#### 사용자 정의 데이터셋(이미지 파일) & 커스텀 데이터셋

- 딥러닝은 대량의 데이터를 이용하여 모델을 학습시킨다.
- 데이터를 한번에 메모리에 불러와서 훈련시키면 시간과 비용 측면에서 비효울적이다.
- 커스텀 데이터셋(custom dataset) : 데이터를 한 번에 다 부리지 않고 조금씩 나누어 불러서 사용하는 방식
- 딥러닝 파이토치 교과서, 서지영 지음(p. 46)

In [1]:
# 이미지 파일을 이용하여 사용자 정의 데이터셋 만들기
# 커스텀 데이터셋 클래스를 구현하기 위해 다음 3개의 함수를 구현한다.
# __init__, __len__, __getitem__

import os 
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms

# import torch
# from torchvision import datasets
# from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [2]:
transform_crop = transforms.CenterCrop((128, 128))   # 지정된 사이즈로 가운데 부분만 크롭
# 사용하는 데이터셋의 이미지의 사이즈가 서로 달라, Dataload 부분에서 에러 발생
# 커스텀 데이터셋 클래스의 메서드에서 전달하는 이미지를 미리 크롭하여 반환하도록 코드 설정(250318)

class CustomImageDataset (Dataset):
    # 라벨 정의 파일 csv, 이미지 저장 폴더
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'])   # 첫번째 컬럼 : 파일명 , 두번째 컬럼 : 라벨
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return transform_crop(image), label    # 이미지 텐서 + 라벨 반환
        # 가운데 크롭된 이미지를 전달 (250318)

In [3]:
# 전달 변수명 일치시킬 것
custom_img_dataset = CustomImageDataset(annotations_file='./img_labels/annotations_file.csv', img_dir='./img_dir')

In [4]:
# 텐서 출력
# enumerate 메서드 : 갯수 + 추출 데이터 반환

for i, sample in enumerate(custom_img_dataset):
    print(i, sample)

0 (tensor([[[205, 203, 200,  ..., 201, 200, 197],
         [204, 202, 202,  ..., 201, 200, 196],
         [202, 202, 200,  ..., 201, 200, 197],
         ...,
         [187, 188, 189,  ..., 195, 196, 196],
         [187, 187, 188,  ..., 196, 197, 198],
         [187, 187, 188,  ..., 197, 197, 199]],

        [[186, 184, 181,  ..., 164, 164, 165],
         [184, 182, 179,  ..., 164, 164, 166],
         [182, 179, 177,  ..., 164, 164, 165],
         ...,
         [189, 188, 189,  ..., 180, 181, 178],
         [189, 189, 190,  ..., 181, 182, 180],
         [189, 189, 189,  ..., 182, 182, 182]],

        [[154, 152, 148,  ..., 112, 114, 116],
         [151, 147, 145,  ..., 112, 112, 114],
         [147, 145, 143,  ..., 112, 112, 114],
         ...,
         [178, 178, 177,  ..., 151, 152, 154],
         [178, 178, 177,  ..., 152, 153, 156],
         [178, 176, 175,  ..., 153, 153, 156]]], dtype=torch.uint8), 'dog')
1 (tensor([[[ 79,  79,  79,  ...,  67,  66,  66],
         [ 78,  78,  78,  

In [5]:
sample[0]

tensor([[[208, 209, 210,  ..., 200, 197, 200],
         [209, 210, 211,  ..., 198, 195, 198],
         [209, 210, 211,  ..., 197, 193, 198],
         ...,
         [174, 206, 160,  ..., 191, 203, 197],
         [194, 221, 185,  ..., 146, 181, 185],
         [188, 187, 184,  ..., 135, 139, 169]],

        [[192, 193, 194,  ..., 184, 181, 184],
         [193, 194, 195,  ..., 182, 179, 182],
         [193, 194, 195,  ..., 181, 177, 182],
         ...,
         [157, 189, 143,  ..., 181, 191, 187],
         [176, 203, 167,  ..., 134, 167, 173],
         [170, 169, 166,  ..., 122, 123, 156]],

        [[179, 180, 181,  ..., 171, 168, 171],
         [180, 181, 182,  ..., 169, 166, 169],
         [180, 181, 182,  ..., 168, 164, 169],
         ...,
         [149, 179, 135,  ..., 169, 179, 175],
         [166, 191, 157,  ..., 120, 154, 159],
         [158, 155, 154,  ..., 106, 110, 140]]], dtype=torch.uint8)

In [6]:
# 텐서 -> 실제 이미지 변환
tf_img = transforms.ToPILImage()
img = tf_img(sample[0])
img.show()

In [7]:
# 이미지와 라벨 표시
from torch.utils.data import DataLoader

train_dataloader = DataLoader(custom_img_dataset, batch_size=3, shuffle=True)

train_feature, train_label = next(iter(train_dataloader))

img = train_feature[1].squeeze()
label = train_label[1]

In [8]:
tf_img = transforms.ToPILImage()
img = tf_img(img)
img.show()

print(f"Label: {label}")

Label: dog


In [9]:
# 파이토치 이미지 처리: https://hands-on.pytorch.kr/object-detection/torchvision-basic-transforms.html